In [2]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv('openai.env')

import datetime as dt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 150)

%load_ext autoreload
%autoreload 2
from back_fill_games import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Get games

In [3]:
res = fetch_games_for_date(dt.date(2023, 12, 12))

DB path: 'basnya.db'
calling GameFinder for 2023-12-12
found games for this date: 5
Found games on latest date of '2023-12-12T00:00:00': 5
Found players: 4902
Searching for games from 12/12/2023 to 12/12/2023
need to append games: 0
no need to append players
writing 0 records to games


In [4]:
res#.sort_values('GAME_ID')

,unseen_game,game_id,game_date,teams,score
0,False,0022300305,12/12/2023,Boston Celtics - Cleveland Cavaliers,120 - 113
1,False,0022300306,12/12/2023,Dallas Mavericks - Los Angeles Lakers,127 - 125
2,False,0022300307,12/12/2023,Chicago Bulls - Denver Nuggets,106 - 114
3,False,0022300308,12/12/2023,Phoenix Suns - Golden State Warriors,119 - 116
4,False,0022300309,12/12/2023,LA Clippers - Sacramento Kings,119 - 99


In [5]:
sorted(get_game_ids_for_date(dt.date(2023, 12, 12)))

calling GameFinder for 2023-12-12
found games for this date: 5


['0022300305', '0022300306', '0022300307', '0022300308', '0022300309']

# Gen tweets by stats

In [6]:
from signal_indicator.resource.prompts_template import prompt
from signal_indicator.resource.strategies import *
from signal_indicator.resource.utils import *

import logging
import os
from pprint import pprint

DB_PATH = 'basnya.db'
selected_game_ids = ['0022300305', '0022300306', '0022300307', '0022300308', '0022300309']

In [7]:
stats = StatsHolder.from_sql(db_path=DB_PATH, prompt=prompt)
strategy_top_10 = StrategyRating(top=10, player_id_to_name_dict=stats.player_id_to_name_dict)
stats.add_strategy(strategy_top_10)

In [8]:
stats.run_strategies(selected_game_ids=selected_game_ids)

{'title': "For ['0022300305', '0022300306', '0022300307', '0022300308', '0022300309'] games",
 'result': [{'description': 'Top 10 players',
   'values': {'in_top': {'MIN': [{'NAME': 'Max Strus', 'COUNTRY': 'USA'}],
     'FGA': [{'NAME': 'LeBron James', 'COUNTRY': 'USA'},
      {'NAME': 'DeMar DeRozan', 'COUNTRY': 'USA'}],
     'FT_PCT': [{'NAME': 'Paul George', 'COUNTRY': 'USA'}],
     'OREB': [{'NAME': 'Nikola Vucevic', 'COUNTRY': 'Montenegro'}],
     'BLK': [{'NAME': 'Dereck Lively II', 'COUNTRY': 'USA'}],
     'PTS': [{'NAME': 'LeBron James', 'COUNTRY': 'USA'},
      {'NAME': 'Anthony Davis', 'COUNTRY': 'USA'}]},
    'out_top': {'MIN': [{'NAME': 'Pascal Siakam', 'COUNTRY': 'Cameroon'}],
     'FGA': [{'NAME': 'Trae Young', 'COUNTRY': 'USA'},
      {'NAME': 'Kyle Kuzma', 'COUNTRY': 'USA'}],
     'FT_PCT': [{'NAME': 'Damian Lillard', 'COUNTRY': 'USA'}],
     'OREB': [{'NAME': 'Ivica Zubac', 'COUNTRY': 'Croatia'}],
     'BLK': [{'NAME': 'Goga Bitadze', 'COUNTRY': 'Georgia'}],
     'PTS'

In [14]:
# openai.api_key  = os.getenv('OPENAI_API_KEY')
stats.get_tweets(selected_game_ids=selected_game_ids)

[Tweet(game_id=None, player_id=None, tweet_text='\n🚨 BREAKING NEWS 🚨 Big changes in the', tweet_type=<TweetType.RATING_CHANGE_SEASON_SUM: 'rating_change_season_sum'>)]

In [31]:
# strategy_top_100 = StrategyRating(top=100, playerID2name=stats.playerId2name)
strategy_top_50 = StrategyRating(top=50, playerID2name=stats.playerId2name)
# strategy_top_10= StrategyRating(top=10, playerID2name=stats.playerId2name)
# stats.add_strategy(strategy_top_10)
stats.add_strategy(strategy_top_50)
# # stats.add_strategy(strategy_top_100)
# # result signal
twits_signal = stats.get_twits(N_last_game=5)

for twit in twits_signal:
    print('===============================================================================================')
    pprint(twit)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.13it/s]
0it [00:00, ?it/s]


# Gen tweets by AD

In [19]:
from anomaly.resource.utils import AnomalyCalculation

In [29]:
anomaly = AnomalyCalculation(db_path=DB_PATH, path_model_weights='isolation_forest_model_2021-10-19_2023-10-23.joblib')

In [37]:
tweets_anomaly = anomaly.get_tweets(selected_game_ids=selected_game_ids)
tweets_anomaly

X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names


use analog: False


[Tweet(game_id=None, player_id=None, tweet_text="\n    Luka Doncic is a beast on the court! With 17 assists and 33 points, he led Dallas to victory. And let's not forget his 12 made shots and 10 three-point attempts. Simply incredible. #MVP #Dallas #Basketball", tweet_type=<TweetType.ANOMALY_UNIQUE: 'anomaly_unique'>)]

In [33]:
%debug

> /home/noone/.virtualenvs/env_83/lib/python3.8/site-packages/shap/explainers/_tree.py(1133)__init__()
   1131             self.base_offset = model.init_params[param_idx]
   1132         else:
-> 1133             raise InvalidModelError("Model type not yet supported by TreeExplainer: " + str(type(model)))
   1134 
   1135         # build a dense numpy version of all the tree objects



ipdb>  u


> /home/noone/.virtualenvs/env_83/lib/python3.8/site-packages/shap/explainers/_tree.py(166)__init__()
    164         self.feature_perturbation = feature_perturbation
    165         self.expected_value = None
--> 166         self.model = TreeEnsemble(model, self.data, self.data_missing, model_output)
    167         self.model_output = model_output
    168         #self.model_output = self.model.model_output # this allows the TreeEnsemble to translate model outputs types by how it loads the model



ipdb>  model


Pipeline(steps=[('scaler', StandardScaler()),
                ('isolation_forest',
                 IsolationForest(contamination=0.01, random_state=42))])


ipdb>  dir(model.steps)


['__add__', '__class__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']


ipdb>  q
